<a href="https://colab.research.google.com/github/aznagyouness/AAA/blob/main/Transfer_learning_with_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)
# cache_dir indicates where to download data. I specified . which means current directory
# untar true will unzip it

In [ ]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('datasets/flower_photos')

In [ ]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

In [ ]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [ ]:
X, y = [], []

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(180,180))
        X.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [ ]:
num_classes = 5

model = Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes) # gives you a linear equation W*a+b, without softmax(it's from logits).
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
model.fit(X_train_scaled, y_train, epochs=30)              

Epoch 1/30
86/86 [==============================] - 11s 25ms/step - loss: 1.3782 - accuracy: 0.4099
Epoch 2/30
86/86 [==============================] - 2s 22ms/step - loss: 1.0028 - accuracy: 0.5999
Epoch 3/30
86/86 [==============================] - 2s 22ms/step - loss: 0.8002 - accuracy: 0.6831
Epoch 4/30
86/86 [==============================] - 2s 24ms/step - loss: 0.5991 - accuracy: 0.7783
Epoch 5/30
86/86 [==============================] - 2s 22ms/step - loss: 0.3953 - accuracy: 0.8597
Epoch 6/30
86/86 [==============================] - 2s 22ms/step - loss: 0.2394 - accuracy: 0.9182
Epoch 7/30
86/86 [==============================] - 2s 22ms/step - loss: 0.1277 - accuracy: 0.9622
Epoch 8/30
86/86 [==============================] - 2s 23ms/step - loss: 0.0783 - accuracy: 0.9789
Epoch 9/30
86/86 [==============================] - 2s 22ms/step - loss: 0.0303 - accuracy: 0.9945
Epoch 10/30
86/86 [==============================] - 2s 22ms/step - loss: 0.0249 - accuracy: 0.9960
Epoch 11

In [ ]:
model.evaluate(X_test_scaled,y_test)

29/29 [==============================] - 1s 16ms/step - loss: 2.3236 - accuracy: 0.6503


[2.3236334323883057, 0.6503267884254456]

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=X[0].shape),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [ ]:
num_classes = 5

model = Sequential([
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
model.fit(X_train_scaled, y_train, epochs=30)    

Epoch 1/30
86/86 [==============================] - 4s 27ms/step - loss: 1.4857 - accuracy: 0.3648
Epoch 2/30
86/86 [==============================] - 2s 25ms/step - loss: 1.1184 - accuracy: 0.5403
Epoch 3/30
86/86 [==============================] - 2s 26ms/step - loss: 1.0160 - accuracy: 0.5992
Epoch 4/30
86/86 [==============================] - 2s 26ms/step - loss: 0.9275 - accuracy: 0.6453
Epoch 5/30
86/86 [==============================] - 2s 26ms/step - loss: 0.8590 - accuracy: 0.6679
Epoch 6/30
86/86 [==============================] - 2s 25ms/step - loss: 0.7936 - accuracy: 0.7057
Epoch 7/30
86/86 [==============================] - 2s 26ms/step - loss: 0.7845 - accuracy: 0.7031
Epoch 8/30
86/86 [==============================] - 2s 26ms/step - loss: 0.7309 - accuracy: 0.7209
Epoch 9/30
86/86 [==============================] - 2s 26ms/step - loss: 0.7051 - accuracy: 0.7300
Epoch 10/30
86/86 [==============================] - 2s 26ms/step - loss: 0.6622 - accuracy: 0.7449
Epoch 11/

In [ ]:
model.evaluate(X_test_scaled,y_test)

29/29 [==============================] - 1s 15ms/step - loss: 1.1448 - accuracy: 0.6895


[1.1447969675064087, 0.6895424723625183]

In [ ]:
m_res = tf.keras.applications.ResNet50V2(weights=None)

In [ ]:
num_classes = 5

model = Sequential([
  data_augmentation,
  tf.keras.layers.Resizing(224, 224),
  m_res,
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)  
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
model.fit(X_train_scaled, y_train, epochs=30)   

Epoch 1/30
86/86 [==============================] - 28s 272ms/step - loss: 1.3859 - accuracy: 0.6414
Epoch 2/30
86/86 [==============================] - 24s 277ms/step - loss: 1.1204 - accuracy: 0.6185
Epoch 3/30
86/86 [==============================] - 24s 282ms/step - loss: 0.9830 - accuracy: 0.6464
Epoch 4/30
86/86 [==============================] - 24s 282ms/step - loss: 0.8873 - accuracy: 0.6886
Epoch 5/30
86/86 [==============================] - 24s 279ms/step - loss: 0.8533 - accuracy: 0.7013
Epoch 6/30
86/86 [==============================] - 24s 280ms/step - loss: 0.8402 - accuracy: 0.7049
Epoch 7/30
86/86 [==============================] - 24s 281ms/step - loss: 0.8208 - accuracy: 0.7024
Epoch 8/30
86/86 [==============================] - 24s 281ms/step - loss: 0.7733 - accuracy: 0.7326
Epoch 9/30
86/86 [==============================] - 24s 280ms/step - loss: 0.7818 - accuracy: 0.7188
Epoch 10/30
86/86 [==============================] - 24s 280ms/step - loss: 0.7531 - accura

In [ ]:
model.evaluate(X_test_scaled,y_test)

29/29 [==============================] - 4s 117ms/step - loss: 1.6302 - accuracy: 0.4717


[1.6301801204681396, 0.4716775715351105]

In [ ]:
m_inception = tf.keras.applications.inception_v3.InceptionV3(weights=None)
num_classes = 5

model = Sequential([
  data_augmentation,
  tf.keras.layers.Resizing(299, 299),
  m_inception,
  layers.Dense(num_classes)  
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
model.fit(X_train_scaled, y_train, epochs=30)   

Epoch 1/30
86/86 [==============================] - 44s 436ms/step - loss: 1.5850 - accuracy: 0.3180
Epoch 2/30
86/86 [==============================] - 37s 434ms/step - loss: 1.5635 - accuracy: 0.3136
Epoch 3/30
86/86 [==============================] - 37s 431ms/step - loss: 1.5349 - accuracy: 0.3630
Epoch 4/30
86/86 [==============================] - 37s 433ms/step - loss: 1.5201 - accuracy: 0.3576
Epoch 5/30
32/86 [==========>...................] - ETA: 23s - loss: 1.4890 - accuracy: 0.3896

KeyboardInterrupt: ignored

In [ ]:

hub_layer = hub.KerasLayer(
    'https://tfhub.dev/google/edgetpu/vision/mobilenet-edgetpu-v2-feature-vector/tiny/2',
    trainable=False)

num_classes = 5
inputs = tf.keras.Input(shape=(224, 224, 3))
x = hub_layer(inputs)
x = tf.keras.layers.Conv2D(num_classes, 1, name='logits')(x)
x = tf.keras.layers.Activation('softmax', name='probs')(x)
outputs = tf.squeeze(x, axis=[1, 2])

model = tf.keras.Model(inputs, outputs)

m_resize = Sequential([
  tf.keras.layers.Resizing(224, 224),
  model  
])

In [ ]:
m_resize.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
              
m_resize.fit(X_train_scaled, y_train, epochs=100)   

Epoch 1/100
86/86 [==============================] - 6s 47ms/step - loss: 0.0921 - accuracy: 0.9840
Epoch 2/100
86/86 [==============================] - 4s 46ms/step - loss: 0.0798 - accuracy: 0.9862
Epoch 3/100
86/86 [==============================] - 4s 46ms/step - loss: 0.0730 - accuracy: 0.9913
Epoch 4/100
86/86 [==============================] - 4s 47ms/step - loss: 0.0647 - accuracy: 0.9916
Epoch 5/100
86/86 [==============================] - 4s 47ms/step - loss: 0.0598 - accuracy: 0.9931
Epoch 6/100
86/86 [==============================] - 4s 47ms/step - loss: 0.0537 - accuracy: 0.9942
Epoch 7/100
86/86 [==============================] - 4s 47ms/step - loss: 0.0497 - accuracy: 0.9956
Epoch 8/100
86/86 [==============================] - 4s 47ms/step - loss: 0.0476 - accuracy: 0.9949
Epoch 9/100
86/86 [==============================] - 4s 47ms/step - loss: 0.0418 - accuracy: 0.9967
Epoch 10/100
86/86 [==============================] - 4s 46ms/step - loss: 0.0398 - accuracy: 0.9971

In [ ]:
m_resize.evaluate(X_test_scaled,y_test)

29/29 [==============================] - 3s 61ms/step - loss: 0.6193 - accuracy: 0.8824


[0.6193068027496338, 0.8823529481887817]

In [ ]:
data_augmentation_transfer = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(224,224,3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

m_resize_augmentation = Sequential([
  tf.keras.layers.Resizing(224, 224),
  data_augmentation_transfer,
  model  
])

m_resize_augmentation.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
              
m_resize_augmentation.fit(X_train_scaled, y_train, epochs=100) 

NameError: ignored

In [ ]:
m_resize_augmentation.evaluate(X_test_scaled, y_test)

29/29 [==============================] - 3s 49ms/step - loss: 0.3853 - accuracy: 0.8791


[0.38525697588920593, 0.8790849447250366]

- FIX the OF essues :

In [ ]:
hub_layer = hub.KerasLayer(
    'https://tfhub.dev/google/edgetpu/vision/mobilenet-edgetpu-v2-feature-vector/tiny/2',
    trainable=False)

num_classes = 5
inputs1 = tf.keras.Input(shape=(224, 224, 3))
x1 = hub_layer(inputs1)
x1 = tf.keras.layers.Conv2D(32, 3, name='logits32', padding='same')(x1) # not forget 'same' else Error
x1 = tf.keras.layers.Dropout(0.5)(x1)
x1 = tf.keras.layers.Conv2D(16, 3, name='logits16', padding='same')(x1) # not forget 'same' else Error
x1 = tf.keras.layers.Dropout(0.5)(x1)
x1 = tf.keras.layers.Conv2D(num_classes, 1, name='logits')(x1)
x1 = tf.keras.layers.Activation('softmax', name='probs')(x1)
outputs1 = tf.squeeze(x1, axis=[1, 2])

model1 = tf.keras.Model(inputs1, outputs1)

m_resize1 = Sequential([
  tf.keras.layers.Resizing(224, 224),
  model1  
])

m_resize1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
              
m_resize1.fit(X_train_scaled, y_train, epochs=30)  

Epoch 1/30
86/86 [==============================] - 13s 47ms/step - loss: 1.0216 - accuracy: 0.6072
Epoch 2/30
86/86 [==============================] - 4s 47ms/step - loss: 0.6630 - accuracy: 0.7584
Epoch 3/30
86/86 [==============================] - 4s 47ms/step - loss: 0.5209 - accuracy: 0.8234
Epoch 4/30
86/86 [==============================] - 4s 47ms/step - loss: 0.4732 - accuracy: 0.8368
Epoch 5/30
86/86 [==============================] - 4s 48ms/step - loss: 0.4322 - accuracy: 0.8543
Epoch 6/30
86/86 [==============================] - 4s 48ms/step - loss: 0.3903 - accuracy: 0.8616
Epoch 7/30
86/86 [==============================] - 4s 48ms/step - loss: 0.3272 - accuracy: 0.8815
Epoch 8/30
86/86 [==============================] - 4s 48ms/step - loss: 0.3084 - accuracy: 0.8950
Epoch 9/30
86/86 [==============================] - 4s 48ms/step - loss: 0.2904 - accuracy: 0.8939
Epoch 10/30
86/86 [==============================] - 4s 49ms/step - loss: 0.2865 - accuracy: 0.9023
Epoch 11

In [ ]:
m_resize1.evaluate(X_test_scaled, y_test)